<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The Riemann Solution on $\tilde{S}_i$ using HLLE
### Author: Patrick Nelson

This notebook documents the function from the original `GiRaFFE` that calculates the flux for $\tilde{S}_i$ according to the the method of Harten, Lax, von Leer, and Einfeldt (HLLE), assuming that we have calculated the values of the velocity and magnetic field on the cell faces according to the piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf), modified for the case of GRFFE. 

**Module Status:** <font color=orange><b> Passing most unit tests </b></font>

**Validation Notes:** Everything should be written at this point, but we need to finish the rest of the algorithm before we can validate this.

The differential equations that `GiRaFFE` evolves are written in conservation form, and thus have two different terms that contribute to the time evolution of some quantity: the flux term and the source term. The PPM method is what the original `GiRaFFE` uses to handle the flux term; hopefully, using this instead of finite-differencing will fix some of the problems we've been having with `GiRaFFE_HO`.

In GRFFE, the evolution equation for the Poynting flux $\tilde{S}_i$ is given as 
$$
\boxed{\partial_t \tilde{S}_i + \underbrace{ \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right)}_{\rm Flux\ term} = \underbrace{\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}}_{\rm Source\ term}.}
$$
We can then see that, if we rewrite this, the right-hand side (RHS) describing the time evolution $\partial_t \tilde{S}_i$ consists of two terms: the flux term and the source term. The flux term in particular can be tricky, as it may be discontinuous due to shocks or other sharp features. This presents difficulties when we take finite-difference derivatives of that term, leading to the Gibbs phenomenon. So, we implement a different algorithm to take the derivative. 

It is the flux term itself that concerns us here; the following functions will compute the value inside the parentheses so that we can easily take its derivative later. Having reconstructed the values of $v^i_{(n)}$ and $B^i$ on the cell faces, we will now compute the value of the flux of $\tilde{S}_i$ on each face. For each component of $\tilde{S}_i$ in each direction, we compute the flux as
$$
{\rm st\_j\_flux} = \frac{c_{\rm min} f_{\rm R} + c_{\rm max} f_{\rm L} - c_{\rm min} c_{\rm max} ({\rm st\_j\_r}-{\rm st\_j\_l})}{c_{\rm min} + c_{\rm max}},
$$
where 
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) u_i + (P+\frac{1}{2}b^2) \delta_{ji} - b^j b_i \right)
$$
and
$$
{\rm st\_j} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 u_i - b^t b_i \right).
$$
Here, $i$ is direction in which we are computing the flux, and $j$ is the component of the momentum we are computing it for. Note that these two quantities are computed on both the left and right sides of the cell face. We will be able to draw heavily on the module [Tutorial-u0_smallb_Poynting-Cartesian.ipynb](Tutorial-u0_smallb_Poynting-Cartesian.ipynb) to compute $u^0$, $u^i$, and $b^\mu$, as well as the index-lowered forms of those vectors. Critically, these quantities depend on the Valencia 3-velocity $v^i_{(n)}$ and magnetic field $B^i$. We will not be using the normal gridfunctions for these, but rather the ones that we have previosly calculated on the left and right sides of the cell faces using the [Piecewise Parabolic Method](Tutorial-GiRaFFE_HO_Ccode_library-PPM.ipynb).

The speeds $c_\min$ and $c_\max$ are characteristic speeds that waves can travel through the plasma. In GRFFE, the expressions defining them reduce a function of only the metric quantities. $c_\min$ is the negative of the minimum amongst the speeds $c_-$ and $0$ and $c_\max$ is the maximum amongst the speeds $c_+$ and $0$. The speeds $c_\pm = \sqrt{b^2-4ac}$ must be calculated on both the left and right faces, where 
$$a = 1/\alpha^2,$$ 
$$b = 2 \beta^i / \alpha^2$$
and $$c = g^{ii} - (\beta^i)^2/\alpha^2.$$

Another point to consider is that since we are working on cell faces, not at the cell center, we can't use the normal metric values that we store. We will instead use the value of the metric interpolated onto the cell face, which in this tutorial, we will assume has been previously done. 

The algorithm for finite-volume methods in general is as follows: 

1. The Reconstruction Step - Piecewise Parabolic Method
    1. Within each cell, fit to a function that conserves the volume in that cell using information from the neighboring cells
        * For PPM, we will naturally use parabolas
    1. Use that fit to define the state at the left and right interface of each cell
    1. Apply a slope limiter to mitigate Gibbs phenomenon
1. Interpolate the value of the metric gridfunctions on the cell faces
1. **Solving the Riemann Problem - Harten, Lax, (This notebook, $\tilde{S}_i$ only)**
    1. **Use the left and right reconstructed states to calculate the unique state at boundary**
1. Use the unique state to estimate the derivative in the cell

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#s_i_flux): The $\tilde{S}_i$ function
    1. [Step 2.a](#define): The function declaration: inputs, outputs, and macros
    1. [Step 2.b](#speed_limit): Speed-limit the velocities
    1. [Step 2.c](#smallb): Computing $b^\mu$, the magnetic field in the comoving fluid frame
    1. [Step 2.d](#hydro_speed): GRFFE characteristic wave speeds
    1. [Step 2.e](#fluxes): Compute the HLLE fluxes
1. [Step 3](#derive_speed): Complete Derivation of the Wave Speeds
1. [Step 4](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just sets up a subdirectory within `GiRaFFE_standalone_Ccodes/` to which we will write the C code. We will also import the core NRPy+ functionality and register the needed gridfunctions. Doing so will let NRPy+ figure out where to read and write data from/to. 

In [1]:
from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions

outdir = "GiRaFFE_standalone_Ccodes/"
cmd.mkdir(outdir)
outdir += "PPM/"
cmd.mkdir(outdir)

par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

thismodule = "GiRaFFE_NRPy_Ccode_library-Stilde-flux"


We will also create the gridfunctions within NRPy+ so that it knows how to access them. Note that we declare different gridfunctions than normal, since we want to use the values of those gridfunctions interpolated on the cell faces.

In [2]:
# These are the standard gridfunctions we've used before.
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","ValenciavU",DIM=3)
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01")
betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU")
alpha = gri.register_gridfunctions("AUXEVOL",["alpha"])
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD",DIM=3)
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU",DIM=3)

# We will pass values of the gridfunction on the cell faces into the function. This requires us
# to declare them as C parameters in NRPy+. We will denote this with the _face infix/suffix.
alpha_face,gammadet_face = gri.register_gridfunctions("AUXEVOL",["alpha_face","gammadet_face"])
gamma_faceDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceDD","sym01")
gamma_faceUU = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceUU","sym01")
beta_faceU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","beta_faceU")

# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
Valenciav_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_rU",DIM=3)
B_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_rU",DIM=3)
Valenciav_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_lU",DIM=3)
B_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_lU",DIM=3)

# ...and some more for the fluxes we calculate here. These three gridfunctions will each store
# the momentum flux of one component of StildeD in one direction; we'll be able to reuse them
# as we loop over each direction, reducing our memory costs.
Stilde_fluxD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Stilde_fluxD",DIM=3)

<a id='s_i_flux'></a>

# Step 2: The $\tilde{S}_i$ function \[Back to [top](#toc)\]
$$\label{s_i_flux}$$

<a id='define'></a>

## Step 2.a: The function declaration: inputs, outputs, and macros \[Back to [top](#toc)\]
$$\label{define}$$

First, we give the function definition. This function will take as parameters the integers `i0`, `i1`, and `i2`; these indicate a point on the grid. The function also needs the array of gridfunctions (which we use here for the reconstructed face values (see [the previous tutorial](Tutorial-GiRaFFE_HO_Ccode_library-PPM.ipynb) and the interpolated metric quantities on the cell interfaces). We also calculate quantities related to the conformal factor $\psi$ (including the square root of the determinant of the metric $\sqrt{\gamma}$) and the lapse $\alpha$; these lines will be handled in Python so the NRPy+ can optimize them. 

In [3]:
# Let's also add some standard NRPy+ macros here. We can also use IDX3 here
# to define and index that we'll keep reusing.
Stilde_flux_string_pre = """
//-----------------------------------------------------------------------------
// Compute the flux for advecting S_i .
//-----------------------------------------------------------------------------
static inline void GRFFE__S_i__flux(const int i0,const int i1,const int i2, REAL *auxevol_gfs) {

  int idx = IDX3(i0,i1,i2);
"""

In [4]:
# Conveniently, we store the metric determinant gamma directly.
alpha_sqrt_gamma = alpha_face*sp.sqrt(gammadet_face)

# We'll also compute some powers of the conformal factor psi:
# Since psi^6 = sqrt(gammadet),
psi = sp.sqrt(gammadet_face)**(1.0/6.0)
psim4 = psi**(-4.0)

In [5]:
# OLD:
#   const REAL psi4 = FACEVAL_LAPSE_PSI4[PSI4];
#   const REAL psi6 = FACEVAL_LAPSE_PSI4[PSI4]*FACEVAL_LAPSE_PSI4[PSI2];
#   const REAL psim4 = 1.0/(psi4);

#   const REAL alpha_sqrt_gamma = FACEVAL_LAPSE_PSI4[LAPSE]*psi6;
#   const REAL ONE_OVER_LAPSE = 1.0/FACEVAL_LAPSE_PSI4[LAPSE];
#   const REAL ONE_OVER_LAPSE_SQUARED=SQR(ONE_OVER_LAPSE);

<a id='speed_limit'></a>

## Step 2.b: Speed-limit the velocities \[Back to [top](#toc)\]
$$\label{speed_limit}$$

Next, we compute the speed-limited velocities on the faces. This function is found in `inlined_functions.C`, however, we will replace it with our own that does the same thing (and is already compatible with the rest of our code). The original function also counts how often the speed had to be limited. We also introduce a variation of the previous speed limiter that allows the process to be incorporated directly into the final expressions that we will pass to `FD_outputC()`.

In [6]:
# Stilde_flux_string_pre += """
#   // Compute face velocities
#   // Begin by computing u0
#   pointwisecalc_u0(idx,auxevol_gfs);
# """

In [7]:
# Stilde_flux_string_pre = """
# #ifndef POINTWISE
# #define POINTWISE
# static inline void pointwisecalc_u0(const int idx,REAL *auxevol_gfs) {
#     // Speed limit on the right face:
#     REAL u0;
#     REAL ValenciavU0 = auxevol_gfs[IDX4pt(VALENCIAV_RU0GF,idx)];
#     REAL ValenciavU1 = auxevol_gfs[IDX4pt(VALENCIAV_RU1GF,idx)];
#     REAL ValenciavU2 = auxevol_gfs[IDX4pt(VALENCIAV_RU2GF,idx)];
#     const REAL alpha = auxevol_gfs[IDX4pt(ALPHA_FACEGF,idx)];
#     const REAL gammaDD00 = auxevol_gfs[IDX4pt(GAMMA_FACEDD00GF,idx)];
#     const REAL gammaDD01 = auxevol_gfs[IDX4pt(GAMMA_FACEDD01GF,idx)];
#     const REAL gammaDD02 = auxevol_gfs[IDX4pt(GAMMA_FACEDD02GF,idx)];
#     const REAL gammaDD11 = auxevol_gfs[IDX4pt(GAMMA_FACEDD11GF,idx)];
#     const REAL gammaDD12 = auxevol_gfs[IDX4pt(GAMMA_FACEDD12GF,idx)];
#     const REAL gammaDD22 = auxevol_gfs[IDX4pt(GAMMA_FACEDD22GF,idx)];

#     {
#     #include "../computeu0_Cfunction.h"
#     }

#     auxevol_gfs[IDX4pt(U4UPPERZERO_RGF,idx)] = u0;
#     auxevol_gfs[IDX4pt(VALENCIAV_RU0GF,idx)] = ValenciavU0;
#     auxevol_gfs[IDX4pt(VALENCIAV_RU1GF,idx)] = ValenciavU1;
#     auxevol_gfs[IDX4pt(VALENCIAV_RU2GF,idx)] = ValenciavU2;
    
#     // Speed limit on the left face:
#     ValenciavU0 = auxevol_gfs[IDX4pt(VALENCIAV_LU0GF,idx)];
#     ValenciavU1 = auxevol_gfs[IDX4pt(VALENCIAV_LU1GF,idx)];
#     ValenciavU2 = auxevol_gfs[IDX4pt(VALENCIAV_LU2GF,idx)];

#     {
#     #include "../computeu0_Cfunction.h"
#     }

#     auxevol_gfs[IDX4pt(U4UPPERZERO_LGF,idx)] = u0;
#     auxevol_gfs[IDX4pt(VALENCIAV_LU0GF,idx)] = ValenciavU0;
#     auxevol_gfs[IDX4pt(VALENCIAV_LU1GF,idx)] = ValenciavU1;
#     auxevol_gfs[IDX4pt(VALENCIAV_LU2GF,idx)] = ValenciavU2;
# }
# #endif

# """ + Stilde_flux_string_pre


<a id='u0_no_if'></a>

### Step 2.b.i: Streamlining $u^0$ \[Back to [top](#toc)\]
$$\label{u0_no_if}$$

When we calculate $u^0$, we first check the Valencia 3-velocity and limit its speed if necessary. This is done as explained in the tutorial found [here](Tutorial-u0_smallb_Poynting-Cartesian.ipynb):

>Then our algorithm for computing $u^0$ is as follows:
>
>If
>$$R=\gamma_{ij}v^i_{(n)}v^j_{(n)}>1 - \frac{1}{\Gamma_{\rm max}^2},$$ 
then adjust the 3-velocity $v^i$ as follows:
>
>$$v^i_{(n)} = \sqrt{\frac{1 - \frac{1}{\Gamma_{\rm max}^2}}{R}}v^i_{(n)}.$$
>
>After this rescaling, we are then guaranteed that if $R$ is recomputed, it will be set to its ceiling value $R=R_{\rm max} = 1 - \frac{1}{\Gamma_{\rm max}^2}$.
>
>Then, regardless of whether the ceiling on $R$ was applied, $u^0$ can be safely computed via
>
>$$
u^0 = \frac{1}{\alpha \sqrt{1-R}}.
$$

However, we can once again eliminate the `if` statement used here to streamline our code generation. Note that when we violate the speed limit, we rescale by a factor $\sqrt{R_{\rm max}/R}$, where $R_{\rm max}=1 - \frac{1}{\Gamma_{\rm max}}$. It thus follows that, by letting $R_\max$ instead equal $\min(R,1 - \frac{1}{\Gamma_{\rm max}})$, we can *always* rescale by that same factor. In the case where we previously would not have rescaled, $\sqrt{R_{\rm max}/R}=1$, and we have not changed anything we didn't want to. 


In [8]:
def compute_u0_noif(gammaDD,alpha,ValenciavU):
    R = sp.sympify(0)
    for i in range(DIM):
        for j in range(DIM):
            R += gammaDD[i][j]*ValenciavU[i]*ValenciavU[j]

    GAMMA_SPEED_LIMIT = par.Cparameters("REAL",thismodule,"GAMMA_SPEED_LIMIT",10.0) # Default value based on
                                                                                    # IllinoisGRMHD.
                                                                                    # GiRaFFE default = 2000.0
    Rmax = 1 - 1/(GAMMA_SPEED_LIMIT*GAMMA_SPEED_LIMIT)
    # Now, we set Rmax = min(Rmax,R):
    # If Rmax>R, then Rmax = 0.5*(Rmax+R-Rmax+R) = R
    # If R>Rmax, then Rmax = 0.5*(Rmax+R+Rmax-R) = Rmax
    Rmax =  sp.Rational(1,2)*(Rmax+R-sp.Abs(Rmax-R))

    rescaledValenciavU = ixp.zerorank1()
    for i in range(DIM):
        rescaledValenciavU[i] = ValenciavU[i]*sp.sqrt(Rmax/R)

    # We stick with the "rescaled" version since we redefine Rmax as detailed above
    rescaledu0 = 1/(alpha*sp.sqrt(1-Rmax))
    return rescaledValenciavU,rescaledu0,sp.sqrt(Rmax/R)


<a id='smallb'></a>

## Step 2.c: Computing $b^\mu$, the magnetic field in the comoving fluid frame \[Back to [top](#toc)\]
$$\label{smallb}$$

Now, we will compute the magnetic field in the comoving fluid frame $b^\mu$ , as defined in [Gammie's paper](https://arxiv.org/pdf/astro-ph/0301509.pdf).  We will also lower the index of the four velocities on the faces $u_\mu$.

In [9]:
# OLD: 
# Stilde_flux_string_pre += """
#   //Next compute b^{\mu}, the magnetic field measured in the comoving fluid frame:
#   const REAL ONE_OVER_LAPSE_SQRT_4PI = ONE_OVER_LAPSE*ONE_OVER_SQRT_4PI;
#   /***********************************************************/
#   /********** RIGHT FACE ************/
#   // Note that smallbr[4] = b^a defined in Gammie's paper, on the right face.
#   REAL u_x_over_u0_psi4r,u_y_over_u0_psi4r,u_z_over_u0_psi4r;
#   REAL smallbr[NUMVARS_SMALLB];
#   // Compute b^{a}, b^2, and u_i over u^0
#   compute_smallba_b2_and_u_i_over_u0_psi4(FACEVAL,FACEVAL_LAPSE_PSI4,Ur,u0_r,ONE_OVER_LAPSE_SQRT_4PI,  
#                                           u_x_over_u0_psi4r,u_y_over_u0_psi4r,u_z_over_u0_psi4r,smallbr);
#   // Then compute u_xr,u_yr, and u_zr. We need to set the zeroth component so we can specify U_LOWER{r,l}[{UX,UY,UZ}] (UX=1,UY=2,UZ=3).
#   const REAL U_LOWERr[4] = { 0.0, u_x_over_u0_psi4r*u0_r*FACEVAL_LAPSE_PSI4[PSI4], u_y_over_u0_psi4r*u0_r*FACEVAL_LAPSE_PSI4[PSI4], 
#                                   u_z_over_u0_psi4r*u0_r*FACEVAL_LAPSE_PSI4[PSI4] };
#   /********** LEFT FACE ************/
#   // Note that smallbl[4] = b^a defined in Gammie's paper, on the left face.
#   REAL u_x_over_u0_psi4l,u_y_over_u0_psi4l,u_z_over_u0_psi4l;
#   REAL smallbl[NUMVARS_SMALLB];
#   // Compute b^{a}, b^2, and u_i over u^0
#   compute_smallba_b2_and_u_i_over_u0_psi4(FACEVAL,FACEVAL_LAPSE_PSI4,Ul,u0_l,ONE_OVER_LAPSE_SQRT_4PI,  
#                                           u_x_over_u0_psi4l,u_y_over_u0_psi4l,u_z_over_u0_psi4l,smallbl);
#   // Then compute u_xr,u_yr, and u_zr. We need to set the zeroth component so we can specify U_LOWER{r,l}[{UX,UY,UZ}]
#   const REAL U_LOWERl[4] = { 0.0, u_x_over_u0_psi4l*u0_l*FACEVAL_LAPSE_PSI4[PSI4], u_y_over_u0_psi4l*u0_l*FACEVAL_LAPSE_PSI4[PSI4], 
#                                   u_z_over_u0_psi4l*u0_l*FACEVAL_LAPSE_PSI4[PSI4] };
#   /***********************************************************/
# """

Conveniently enough, the NRPy+ code that does this has already been written, and is found in `u0_smallb_Poynting__Cartesian.py`, which is documented [here](Tutorial-u0_smallb_Poynting-Cartesian.ipynb). We will call the function at least twice, once each for left and right side of the face. If we use the old version of the speed-limiting algorithm, we will need to call it a third time so that we generate a more generic version of `computeu0_Cfunction.h`.

In [10]:
# We already did something like this in GiRaFFE_Higher_Order_v2; we'll 
# paste that code here, but using the face variables. This assumes that
# we've already interpolated the metric quantities on the faces.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0b

u0b.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BU)
with open(os.path.join(outdir,"..","computeu0_Cfunction.h"), "w") as file:
    file.write(u0b.computeu0_Cfunction)

Valenciav_rU,u4upperZero_r,R_r = compute_u0_noif(gamma_faceDD,alpha_face,Valenciav_rU)
Valenciav_lU,u4upperZero_l,R_l = compute_u0_noif(gamma_faceDD,alpha_face,Valenciav_lU)

# u4upperZero_l = gri.register_gridfunctions("AUXEVOL","u4upperZero_l")
# u4upperZero_r = gri.register_gridfunctions("AUXEVOL","u4upperZero_r")
# LEFT
u0b.compute_u0_smallb_Poynting__Cartesian(gamma_faceDD,beta_faceU,alpha_face,Valenciav_lU,B_lU)
u_lD = ixp.zerorank1()
u_lU = ixp.zerorank1()

for i in range(DIM):
    u_lD[i] = u0b.uD[i].subs(u0b.u0,u4upperZero_l)
    u_lU[i] = u0b.uU[i].subs(u0b.u0,u4upperZero_l)

smallb4_lU = ixp.zerorank1(DIM=4)
smallb4_lD = ixp.zerorank1(DIM=4)
for mu in range(4):
    smallb4_lU[mu] = u0b.smallb4U[mu].subs(u0b.u0,u4upperZero_l)
    smallb4_lD[mu] = u0b.smallb4D[mu].subs(u0b.u0,u4upperZero_l)

smallb2_l = u0b.smallb2etk.subs(u0b.u0,u4upperZero_l)

# RIGHT
u0b.compute_u0_smallb_Poynting__Cartesian(gamma_faceDD,beta_faceU,alpha_face,Valenciav_rU,B_rU)
u_rD = ixp.zerorank1()
u_rU = ixp.zerorank1()

for i in range(DIM):
    u_rD[i] = u0b.uD[i].subs(u0b.u0,u4upperZero_r)
    u_rU[i] = u0b.uU[i].subs(u0b.u0,u4upperZero_r)

smallb4_rU = ixp.zerorank1(DIM=4)
smallb4_rD = ixp.zerorank1(DIM=4)
for mu in range(4):
    smallb4_rU[mu] = u0b.smallb4U[mu].subs(u0b.u0,u4upperZero_r)
    smallb4_rD[mu] = u0b.smallb4D[mu].subs(u0b.u0,u4upperZero_r)

smallb2_r = u0b.smallb2etk.subs(u0b.u0,u4upperZero_r)


<a id='hydro_speed'></a>

## Step 2.d: GRFFE characteristic wave speeds \[Back to [top](#toc)\]
$$\label{hydro_speed}$$

Next, we will find the speeds at which the hydrodynamics waves propagate. We start from the speed of light (since FFE deals with very diffuse plasmas), which is $c=1.0$ in our chosen units. We then find the speeds $c_+$ and $c_-$ on each face with the function `find_cp_cm`; then, we find minimum and maximum speeds possible from among those.

Below is the source code for `find_cp_cm`:

```c
static inline void find_cp_cm(CCTK_REAL &cplus,CCTK_REAL &cminus,const CCTK_REAL v02,const CCTK_REAL u0,
                              const CCTK_REAL vi,const CCTK_REAL ONE_OVER_LAPSE_SQUARED,const CCTK_REAL shifti,
                              const CCTK_REAL psim4,const CCTK_REAL gupii) {
  const CCTK_REAL u0_SQUARED=SQR(u0);

  //Find cplus, cminus:
  const CCTK_REAL a = u0_SQUARED * (1.0-v02) + v02*ONE_OVER_LAPSE_SQUARED;
  const CCTK_REAL b = 2.0* ( shifti*ONE_OVER_LAPSE_SQUARED * v02 - u0_SQUARED * vi * (1.0-v02) );
  const CCTK_REAL c = u0_SQUARED*SQR(vi) * (1.0-v02) - v02 * ( psim4*gupii -
                                                               SQR(shifti)*ONE_OVER_LAPSE_SQUARED);
  CCTK_REAL detm = b*b - 4.0*a*c;
  //ORIGINAL LINE OF CODE:
  //if(detm < 0.0) detm = 0.0;
  //New line of code (without the if() statement) has the same effect:
  detm = sqrt(0.5*(detm + fabs(detm))); /* Based on very nice suggestion from Roland Haas */
  
  cplus = 0.5*(detm-b)/a;
  cminus = -0.5*(detm+b)/a;
  if (cplus < cminus) {
    const CCTK_REAL cp = cminus;
    cminus = cplus;
    cplus = cp;
  }
}
```
Comments documenting this have been excised for brevity, but are reproduced in $\LaTeX$ [below](#derive_speed).

What follows here is an edited version adapted to the NRPy+ version of GiRaFFE. One edit we need to make in particular is to the term `psim4*gupii` in the definition of `c`; that was written assuming the use of the conformal metric $\tilde{g}^{ii}$. Since we are not using that here, and are instead using the ADM metric, we should not multiply by $\psi^{-4}$.

```c
static inline void find_cp_cm(REAL &cplus,REAL &cminus,const REAL v02,const REAL u0,
                              const REAL vi,const REAL lapse,const REAL shifti,
                              const REAL gammadet,const REAL gupii) {
  const REAL u0_SQUARED=u0*u0;
  const REAL ONE_OVER_LAPSE_SQUARED = 1.0/(lapse*lapse);
  // sqrtgamma = psi6 -> psim4 = gammadet^(-1.0/3.0)
  const REAL psim4 = pow(gammadet,-1.0/3.0);
  //Find cplus, cminus:
  const REAL a = u0_SQUARED * (1.0-v02) + v02*ONE_OVER_LAPSE_SQUARED;
  const REAL b = 2.0* ( shifti*ONE_OVER_LAPSE_SQUARED * v02 - u0_SQUARED * vi * (1.0-v02) );
  const REAL c = u0_SQUARED*vi*vi * (1.0-v02) - v02 * ( gupii -
                                                               shifti*shifti*ONE_OVER_LAPSE_SQUARED);
  REAL detm = b*b - 4.0*a*c;
  //ORIGINAL LINE OF CODE:
  //if(detm < 0.0) detm = 0.0;
  //New line of code (without the if() statement) has the same effect:
  detm = sqrt(0.5*(detm + fabs(detm))); /* Based on very nice suggestion from Roland Haas */
  
  cplus = 0.5*(detm-b)/a;
  cminus = -0.5*(detm+b)/a;
  if (cplus < cminus) {
    const REAL cp = cminus;
    cminus = cplus;
    cplus = cp;
  }
}
```
But there's substantial improvement we can make by changing the the code into a NRPyfied form. Note the `if` statement; NRPy+ does not know how to handle them, so we must eliminate it if we want to leverage NRPy+'s full power. (Calls to `fabs()` are also cheaper than `if` statements.) This can be done if we rewrite this like so:
```c
  cp = 0.5*(detm-b)/a;
  cm = -0.5*(detm+b)/a;
  cplus  = 0.5*(cp+cm+fabs(cp-cm));
  cminus = 0.5*(cp+cm-fabs(cp-cm));
```
This can be simplified further, by substituting `cp` and `cm` into the below equations and eliminating terms as appropriate. First note that `cp+cm = -b/a` and that `cp-cm = detm/a`. Thus,
```c
  cplus  = 0.5*(-b/a + fabs(detm/a));
  cminus = 0.5*(-b/a - fabs(detm/a));
```
This fulfills the original purpose of the `if` statement in the original code because we have guaranteed that $c_+ \geq c_-$.

This leaves us with an expression that can be much more easily NRPyfied. So, we will rewrite the following in NRPy+, making only minimal changes to be proper Python. However, it turns out that we can make this even simpler. In GRFFE, $v_0^2$ is guaranteed to be exactly one. In GRMHD, this speed was calculated as $$v_{0}^{2} = v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right),$$ where the Alfv&eacute;n speed $v_{\rm A}^{2}$ $$v_{\rm A}^{2} = \frac{b^{2}}{\rho_{b}h + b^{2}}.$$ So, we can see that when the density $\rho_b$ goes to zero, $v_{0}^{2} = v_{\rm A}^{2} = 1$. Then 
\begin{align}
a &= (u^0)^2 (1-v_0^2) + v_0^2/\alpha^2 \\
&= 1/\alpha^2 \\
b &= 2 \left(\beta^i v_0^2 / \alpha^2 - (u^0)^2 v^i (1-v_0^2)\right) \\
&= 2 \beta^i / \alpha^2 \\
c &= (u^0)^2 (v^i)^2 (1-v_0^2) - v_0^2 \left(g^{ii} - (\beta^i)^2/\alpha^2\right) \\
&= g^{ii} - (\beta^i)^2/\alpha^2,
\end{align}
are simplifications that should save us some time; we can see that $a \geq 0$ is guaranteed. Note that we also force `detm` to be positive. Thus, `detm/a` is guaranteed to be positive itself, rendering the calls to `sp.Abs()` superfluous. Furthermore, we eliminate any dependence on the Valencia 3-velocity and the time compoenent of the four-velocity, $u^0$. This leaves us free to solve the quadratic in the familiar way: $$c_\pm = \frac{-b \pm \sqrt{b^2-4ac}}{2a}$$.

In [11]:
# We'll write this as a function so that we can calculate the expressions on-demand for any choice of i
def find_cp_cm(lapse,shifti,gupii):
    # Inputs:  u0,vi,lapse,shift,gammadet,gupii
    # Outputs: cplus,cminus 
    a = 1/(lapse*lapse)
    b = 2 * shifti /(lapse*lapse)
    c = - gupii + shifti*shifti/(lapse*lapse)
    detm = b*b - 4*a*c
    detm = sp.sqrt(sp.Rational(1,2)*(detm + sp.Abs(detm)))
    cplus  = sp.Rational(1,2)*(-b/a + detm/a)
    cminus = sp.Rational(1,2)*(-b/a - detm/a)
    return cplus,cminus


In [12]:
# Deprecated:
# Stilde_flux_string_pre = """
# static inline void find_cp_cm(REAL &cplus,REAL &cminus,const REAL v02,const REAL u0,
#                               const REAL vi,const REAL lapse,
#                               const REAL shifti,const REAL gammadet,
#                               const REAL gupii) {
#   const REAL u0_SQUARED=u0*u0;
#   const REAL ONE_OVER_LAPSE_SQUARED = 1.0/(lapse*lapse);
#   // sqrtgamma = psi6 -> psim4 = gammadet^(-1.0/3.0)
#   const REAL psim4 = pow(gammadet,-1.0/3.0);
#   //Find cplus, cminus:
#   const REAL a = u0_SQUARED * (1.0-v02) + v02*ONE_OVER_LAPSE_SQUARED;
#   const REAL b = 2.0* ( shifti*ONE_OVER_LAPSE_SQUARED * v02 - u0_SQUARED * vi * (1.0-v02) );
#   const REAL c = u0_SQUARED*vi*vi * (1.0-v02) - v02 * ( psim4*gupii -
#                                                                shifti*shifti*ONE_OVER_LAPSE_SQUARED);
#   REAL detm = b*b - 4.0*a*c;
#   //ORIGINAL LINE OF CODE:
#   //if(detm < 0.0) detm = 0.0;
#   //New line of code (without the if() statement) has the same effect:
#   detm = sqrt(0.5*(detm + fabs(detm))); /* Based on very nice suggestion from Roland Haas */
  
#   cplus = 0.5*(detm-b)/a;
#   cminus = -0.5*(detm+b)/a;
#   if (cplus < cminus) {
#     const REAL cp = cminus;
#     cminus = cplus;
#     cplus = cp;
#   }
# }
# """ + Stilde_flux_string_pre


Below is the code that the old `GiRaFFE` used to call that function; we will write a similar function in NRPy+, allowing us to generate the expressions with less need to copy-and-paste code. Notice that since we eliminated the dependence on velocities, none of the input quantities are different on either side of the face. So, this function won't really do much besides guarantee that `cmax` and `cmin` are positive and negative, respectively, but we'll leave the machinery here since it is likely to be a useful guide to somebody who wants to something similar. We use the same technique as above to replace the `if` statements inherent to the `MAX()` and `MIN()` functions.

In [13]:
# OLD: 
#   int offset=flux_dirn-1;

#   CCTK_REAL cplusr,cminusr,cplusl,cminusl;
#   find_cp_cm(cplusr,cminusr,v02r,u0_r,
#              Ur[VX+offset],ONE_OVER_LAPSE_SQUARED,FACEVAL[SHIFTX+offset],psim4,FACEVAL[GUPXX+offset]);
#   find_cp_cm(cplusl,cminusl,v02l,u0_l,
#              Ul[VX+offset],ONE_OVER_LAPSE_SQUARED,FACEVAL[SHIFTX+offset],psim4,FACEVAL[GUPXX+offset]);

#   // Then compute cmax, cmin. This is required for the HLL flux.
#   const CCTK_REAL cmaxL =  MAX(0.0,MAX(cplusl,cplusr));
#   const CCTK_REAL cminL = -MIN(0.0,MIN(cminusl,cminusr));

In [14]:
# We'll write this as a function, and call it within HLLE_solver, below.
def find_cmax_cmin(flux_dirn,gamma_faceUU,beta_faceU,alpha_face,gammadet_face):
    # First, we need to find the characteristic speeds on each face
    cpr,cmr = find_cp_cm(alpha_face,beta_faceU[flux_dirn],gamma_faceUU[flux_dirn][flux_dirn])
    cpl,cml = find_cp_cm(alpha_face,beta_faceU[flux_dirn],gamma_faceUU[flux_dirn][flux_dirn])
    
    # The following algorithms have been verified with random floats
    # Now, we need to set cmax to the larger of cpr,cpl, and 0
    cmax = sp.Rational(1,2)*(cpr+cpl+sp.Abs(cpr-cpl))
    cmax = sp.Rational(1,2)*(cmax+sp.Abs(cmax))
    
    # And then, set cmin to the smaller of cmr,cml, and 0
    cmin =  sp.Rational(1,2)*(cmr+cml-sp.Abs(cmr-cml))
    cmin = -sp.Rational(1,2)*(cmin-sp.Abs(cmin))
    return cmax,cmin

# for flux_dirn in range(3):
#     cmax,cmin = find_cmax_cmin(flux_dirn,gamma_faceUU,beta_faceU,alpha_face,gammadet_face)
#     outputC(cmax,"cmax",filename="stdout",params="outCverbose=False")

**Former Step 2.e: Some final needed quantities (Not needed in NRPy)**

Now, we lower the index on the spatial parts of $b^\mu$, $b^i$. Since $b_\nu = g_{\mu\nu} b^\mu$ and 
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_j \\
\beta_i & \gamma_{ij}
\end{pmatrix},
$$
it follows that $b_j = g_{ij}(b^i + b^t \beta^i)$. We calculate this on both the left and right face of the cell.

(In the NRPy+ version, this step was already handled when we called `compute_u0_smallb_Poynting__Cartesian()`.)

In [15]:
# Stilde_flux_string_pre += """
#   //*********************************************************************
#   // momentum flux = \alpha \sqrt{\gamma} T^m_j, where m is the current flux direction (the m index)
#   //*********************************************************************
#   // b_j = g_{ij} (b^i + b^t shift^i), g_{ij} = physical metric
#   //REAL sbtr=0,sbtl=0;
#   REAL smallb_lowerr[NUMVARS_SMALLB],smallb_lowerl[NUMVARS_SMALLB];
#   lower_4vector_output_spatial_part(psi4,FACEVAL,smallbr,smallb_lowerr);
#   lower_4vector_output_spatial_part(psi4,FACEVAL,smallbl,smallb_lowerl);
# """

In [16]:
# This was done above; we could do it here, but we would need to add two
# extra function calls to u0b.compute_u0_smallb_Poynting__Cartesian()

We will also compute some useful hydrodynamics quantities on each face. These are simplified by the fact that pressure and density are both zero in FFE: 
\begin{align}
\rho + b^2 &= b^2 \\
P + \frac{1}{2} b^2 &= \frac{1}{2} b^2\\
\end{align}

In [17]:
# Stilde_flux_string_pre += """
#   /********** Flux for S_x **********/
#   // [S_x flux] = \alpha \sqrt{\gamma} T^m_x, where m is the current flux direction (the m index)
#   //    Again, offset = 0 for reconstruction in x direction, 1 for y, and 2 for z
#   //    Note that kronecker_delta[flux_dirn][0] = { 1 if flux_dirn==1, 0 otherwise }.
#   /********** RIGHT FACE ************/
#   // Compute a couple useful hydro quantities:
#   const REAL rho0_h_plus_b2_r = smallbr[SMALLB2];     // Since rho=0 in GRFFE
#   const REAL P_plus_half_b2_r = 0.5*smallbr[SMALLB2]; // Since P=0 in GRFFE
#   /********** LEFT FACE *************/
#   // Compute a couple useful hydro quantities:
#   const REAL rho0_h_plus_b2_l = smallbl[SMALLB2];     // Since rho=0 in GRFFE
#   const REAL P_plus_half_b2_l = 0.5*smallbl[SMALLB2]; // Since P=0 in GRFFE
# """

In [18]:
# I think it's safe to skip this in GRFFE. We'll leave it in the documentation
# to make it easier to convert to full MHD.

<a id='fluxes'></a>

## Step 2.e: Compute the HLLE fluxes \[Back to [top](#toc)\]
$$\label{fluxes}$$

Finally, we can compute the flux in each direction. This momentum flux in the $m$ direction is defined as $\alpha \sqrt{\gamma} T^m_{\ \ j}$. We have already defined $\alpha \sqrt{\gamma}$, so all we need to do is calculate $T^m_{\ \ j}$. We will do so in accordance with the method published by [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002)  and [Einfeldt](https://epubs.siam.org/doi/10.1137/0725021) (hereafter HLLE) to solve the Riemann problem. So, we define $f(u) = T^m_{\ \ j}$ on each face as 
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) u_x + (P+\frac{1}{2}b^2) \delta_{jx} - b^j b_x \right);
$$
we use $j$ to correspond to the direction in which we are calculating the flux, based on the input `flux_dirn`; that is, $j=1$ corresponds to $x$, and so forth. $\delta_{ij}$ is the standard Kronecker delta. Here, $v^j$ is the drift velocity $u^j/u^0$, which will offer a minor simplification, because $u^0 v^j = u^j$. We also define `st_x_r` and `st_x_l` (called $u_{\rm R}$ and $u_{\rm L}$ in HLL; we'll not do that, since we're using $u$ for four-velocity):
$$
{\rm st\_x} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 u_x - b^t b_x \right)
$$
and combine based on eq. 3.15 in the HLLE paper,
$$
{\rm st\_x\_flux} = \frac{c_{\rm min} f_{\rm R} + c_{\rm max} f_{\rm L} - c_{\rm min} c_{\rm max} ({\rm st\_x\_r}-{\rm st\_x\_l})}{c_{\rm min} + c_{\rm max}}
$$

In [19]:
# Stilde_flux_string_pre += """
#   REAL Fr = alpha_sqrt_gamma*( rho0_h_plus_b2_r*(u0_r*Ur[VX+offset])*U_LOWERr[UX] 
#                                     + P_plus_half_b2_r*kronecker_delta[flux_dirn][0] - smallbr[SMALLBX+offset]*smallb_lowerr[SMALLBX] );
#   REAL Fl = alpha_sqrt_gamma*( rho0_h_plus_b2_l*(u0_l*Ul[VX+offset])*U_LOWERl[UX] 
#                                     + P_plus_half_b2_l*kronecker_delta[flux_dirn][0] - smallbl[SMALLBX+offset]*smallb_lowerl[SMALLBX] );

#   //        S_x =\alpha\sqrt{\gamma}( T^0_x )
#   const REAL st_x_r = alpha_sqrt_gamma*( rho0_h_plus_b2_r*u0_r*U_LOWERr[UX] - smallbr[SMALLBT]*smallb_lowerr[SMALLBX] );
#   const REAL st_x_l = alpha_sqrt_gamma*( rho0_h_plus_b2_l*u0_l*U_LOWERl[UX] - smallbl[SMALLBT]*smallb_lowerl[SMALLBX] );

#   // HLL step for Sx:
#   st_x_flux = (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_x_r-st_x_l) )/(cmaxL + cminL);
# """

Several cells farther down are dedicated to repeating the above in the y and z directions. However, with the power of NRPy+, we will loop over `flux_dirn` and write three versions of the flux function. We'll write the HLLE step as a function so that we can write it alongside its documentation and corresonding code from the original `GiRaFFE`, but call it multiple times to fill out the entire code.

In [20]:
# Note the Kronecker delta symbol in the above; let's create it real quick:
# We'll  zero-offset everything here, unlike in original GiRaFFE.
kronecker_delta = ixp.zerorank2()
for i in range(DIM): 
    kronecker_delta[i][i] = 1

# We'll rewrite this assuming that we've passed the entire reconstructed
# gridfunctions. You could also do this with only one point, but then you'd 
# need to declare everything as a Cparam in NRPy+

def HLLE_solver(flux_dirn, mom_comp): 
    # This solves the Riemann problem for the mom_comp component of the momentum
    # flux StildeD in the flux_dirn direction.
    Fr = alpha_sqrt_gamma * (smallb2_r * u_rU[flux_dirn] * u_rD[mom_comp] 
                             + smallb2_r * kronecker_delta[flux_dirn][mom_comp] / 2.0
                             - smallb4_rU[flux_dirn+1] * smallb4_rD[mom_comp+1])
    
    Fl = alpha_sqrt_gamma * (smallb2_l * u_lU[flux_dirn] * u_lD[mom_comp] 
                             + smallb2_l * kronecker_delta[flux_dirn][mom_comp] / 2.0
                             - smallb4_lU[flux_dirn+1] * smallb4_lD[mom_comp+1])
    
    st_j_r = alpha_sqrt_gamma * (smallb2_r*u4upperZero_r*u_rD[mom_comp] - smallb4_rU[0]*smallb4_rD[mom_comp+1])
    st_j_l = alpha_sqrt_gamma * (smallb2_l*u4upperZero_l*u_lD[mom_comp] - smallb4_lU[0]*smallb4_lD[mom_comp+1])
    
    cmaxL,cminL = find_cmax_cmin(flux_dirn,gamma_faceUU,beta_faceU,alpha_face,gammadet_face)
    
#     return smallb4_rU[mom_comp+1]
    return (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_j_r-st_j_l) )/(cmaxL + cminL)


Finally, now that we have the equations written, we can write out the files. After writing a brief post string (to close the function), we will loop over `flux_dirn`, creating one C file for each flux direction. In each file, we will include the math to calculate each momentum-flux component `mom_comp` in that direction.

We have written the function `HLLE_solver()` so that we can easily compute the flux as specified by those two indices.

In [21]:
# We'll need a post-string as well to close the function.
Stilde_flux_string_post = """
}
"""

# And now, we'll write the files
for flux_dirn in range(DIM):
    out_string = Stilde_flux_string_pre.replace("GRFFE__S_i__flux","GRFFE__S_"+str(flux_dirn)+"__flux")
    for mom_comp in range(DIM):
        Stilde_fluxD[mom_comp] = HLLE_solver(flux_dirn,mom_comp)
    Stilde_flux_to_print   = [\
                              lhrh(lhs=gri.gfaccess("out_gfs","Stilde_fluxD0"),rhs=Stilde_fluxD[0]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","Stilde_fluxD1"),rhs=Stilde_fluxD[1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","Stilde_fluxD2"),rhs=Stilde_fluxD[2]),\
                             ]
    Stilde_flux_kernel = fin.FD_outputC("returnstring",Stilde_flux_to_print,params="outCverbose=False")
    out_string += Stilde_flux_kernel
    out_string += Stilde_flux_string_post
#     Comment out for now: output directory doesn't exist.
    with open(os.path.join(outdir,"GRFFE__S_"+str(flux_dirn)+"__flux.C"), "w") as file:
        file.write(out_string)



We repeat this step in the $y$ direction:
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) u_y + (P+\frac{1}{2}b^2) \delta_{jy} - b^j b_y \right);
$$
and 
$$
{\rm st\_y} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 u_y - b^t b_y \right),
$$
combined as
$$
{\rm st\_y\_flux} = \frac{c_{\rm min} f_{\rm R} + c_{\rm max} f_{\rm L} - c_{\rm min} c_{\rm max} ({\rm st\_y\_r}-{\rm st\_y\_l})}{c_{\rm min} + c_{\rm max}}
$$

In [22]:
# OLD:
#   /********** Flux for S_y **********/
#   // [S_y flux] = \alpha \sqrt{\gamma} T^m_y, where m is the current flux direction (the m index)
#   //    Again, offset = 1 for reconstruction in x direction, 2 for y, and 3 for z
#   //    Note that kronecker_delta[flux_dirn][1] = { 1 if flux_dirn==2, 0 otherwise }.
#   Fr = alpha_sqrt_gamma*( rho0_h_plus_b2_r*(u0_r*Ur[VX+offset])*U_LOWERr[UY] + P_plus_half_b2_r*kronecker_delta[flux_dirn][1] 
#                           - smallbr[SMALLBX+offset]*smallb_lowerr[SMALLBY] );
#   Fl = alpha_sqrt_gamma*( rho0_h_plus_b2_l*(u0_l*Ul[VX+offset])*U_LOWERl[UY] + P_plus_half_b2_l*kronecker_delta[flux_dirn][1] 
#                           - smallbl[SMALLBX+offset]*smallb_lowerl[SMALLBY] );

#   //        S_y =\alpha\sqrt{\gamma}( T^0_y )
#   const REAL st_y_r = alpha_sqrt_gamma*( rho0_h_plus_b2_r*u0_r*U_LOWERr[UY] - smallbr[SMALLBT]*smallb_lowerr[SMALLBY] );
#   const REAL st_y_l = alpha_sqrt_gamma*( rho0_h_plus_b2_l*u0_l*U_LOWERl[UY] - smallbl[SMALLBT]*smallb_lowerl[SMALLBY] );

#   // HLL step for Sy:
#   st_y_flux = (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_y_r-st_y_l) )/(cmaxL + cminL);


and in z:
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) u_z + (P+\frac{1}{2}b^2) \delta_{jz} - b^j b_z \right);
$$
and 
$$
{\rm st\_z} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 uv_z - b^t b_z \right),
$$
combined as
$$
{\rm st\_z\_flux} = \frac{c_{\rm min} f_{\rm R} + c_{\rm max} f_{\rm L} - c_{\rm min} c_{\rm max} ({\rm st\_z\_r}-{\rm st\_z\_l})}{c_{\rm min} + c_{\rm max}}
$$

In [23]:
# OLD:
# /********** Flux for S_z **********/
#   // [S_z flux] = \alpha \sqrt{\gamma} T^m_z, where m is the current flux direction (the m index)
#   //    Again, offset = 1 for reconstruction in x direction, 2 for y, and 3 for z
#   //    Note that kronecker_delta[flux_dirn][2] = { 1 if flux_dirn==3, 0 otherwise }.
#   Fr = alpha_sqrt_gamma*( rho0_h_plus_b2_r*(u0_r*Ur[VX+offset])*U_LOWERr[UZ] + P_plus_half_b2_r*kronecker_delta[flux_dirn][2] 
#                           - smallbr[SMALLBX+offset]*smallb_lowerr[SMALLBZ] );
#   Fl = alpha_sqrt_gamma*( rho0_h_plus_b2_l*(u0_l*Ul[VX+offset])*U_LOWERl[UZ] + P_plus_half_b2_l*kronecker_delta[flux_dirn][2] 
#                           - smallbl[SMALLBX+offset]*smallb_lowerl[SMALLBZ] );

#   //        S_z =\alpha\sqrt{\gamma}( T^0_z )
#   const REAL st_z_r = alpha_sqrt_gamma*( rho0_h_plus_b2_r*u0_r*U_LOWERr[UZ] - smallbr[SMALLBT]*smallb_lowerr[SMALLBZ] );
#   const REAL st_z_l = alpha_sqrt_gamma*( rho0_h_plus_b2_l*u0_l*U_LOWERl[UZ] - smallbl[SMALLBT]*smallb_lowerl[SMALLBZ] );

#   // HLL step for Sz:
#   st_z_flux = (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_z_r-st_z_l) )/(cmaxL + cminL);

#   cmax = cmaxL;
#   cmin = cminL;
# }


<a id='derive_speed'></a>

# Step 3: Complete Derivation of the Wave Speeds \[Back to [top](#toc)\]
$$\label{derive_speed}$$

This computes phase speeds in the direction given by flux_dirn. Note that we replace the full dispersion relation with a simpler one, which overestimates the maximum speeds by a factor of ~2. See full discussion around Eqs. 49 and 50 in [Duez, et al.](http://arxiv.org/pdf/astro-ph/0503420.pdf). In summary, we solve the dispersion relation (in, e.g., the $x$-direction) with a wave vector of $k_\mu = (-\omega,k_x,0,0)$. So, we solve the approximate dispersion relation $\omega_{\rm cm}^2 = [v_A^2 + c_s^2 (1-v_A^2)]k_{\rm cm}^2$ for the wave speed $\omega/k_x$, where the sound speed $c_s = \sqrt{\Gamma P/(h \rho_0)}$, the Alfv&eacute;n speed $v_A = 1$ (in GRFFE), $\omega_{\rm cm} = -k_\mu k^\mu$ is the frequency in the comoving frame, $k_{\rm cm}^2 = K_\mu K^\mu$ is the wavenumber squared in the comoving frame, and $K_\mu = (g_{\mu\nu} + u_\mu u_\nu)k^\nu$ is the part of the wave vector normal to the four-velocity $u^\mu$. See below for a complete derivation.

What follows is a complete derivation of the quadratic we solve.
\begin{align}
w_{\rm cm} &= (-k_0 u^0 - k_x u^x) \\
k_{\rm cm}^2 &= K_{\mu} K^{\mu}, \\
K_{\mu} K^{\mu} &= (g_{\mu a} + u_{\mu} u_a) k^a  g^{\mu b} [ (g_{c b} + u_c u_b) k^c ] \\
\rightarrow g^{\mu b} (g_{c b} + u_{c} u_{b}) k^c &= (\delta^{\mu}_c + u_c u^{\mu} ) k^c \\
                 &= (g_{\mu a} + u_{\mu} u_a) k^a  (\delta^{\mu}_c + u_c u^{\mu} ) k^c \\
                 &=[(g_{\mu a} + u_{\mu} u_a) \delta^{\mu}_c + (g_{\mu a} + u_{\mu} u_a) u_c u^{\mu} ] k^c k^a \\
                 &=[(g_{c a} + u_c u_a) + (u_c u_a -  u_a u_c] k^c k^a \\
                 &=(g_{c a} + u_c u_a) k^c k^a \\
                 &= k_a k^a + u^c u^a k_c k_a \\
k^a = g^{\mu a} k_{\mu} &= g^{0 a} k_0 + g^{x a} k_x \\
k_a k^a &= k_0 g^{0 0} k_0 + k_x k_0 g^{0 x} + g^{x 0} k_0 k_x + g^{x x} k_x k_x \\
         &= g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2 \\
u^c u^a k_c k_a &= (u^0 k_0 + u^x k_x) (u^0 k_0 + u^x k_x) = (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 \\
(k_0 u0)^2  + 2 k_x u^x k_0 u^0 + (k_x u^x)^2 &= v_0^2 [ (u^0 k_0)^2 + 2 u^x k_x u^0 k_0 + (u^x k_x)^2 + g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2] \\
(1-v_0^2) (u^0 k_0 + u^x k_x)^2 &= v_0^2 (g^{00} (k_0)^2 + 2 g^{x0} k_0 k_x + g^{xx} (k_x)^2) \\
(1-v_0^2) (u^0 k_0/k_x + u^x)^2 &= v_0^2 (g^{00} (k_0/k_x)^2 + 2 g^{x0} k_0/k_x + g^{xx}) \\
(1-v_0^2) (u^0 X + u^x)^2 &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
(1-v_0^2) ((u^0)^2 X^2 + 2 u^x (u^0) X + (u^x)^2) &= v_0^2 (g^{00} X^2 + 2 g^{x0} X + g^{xx}) \\
&X^2 ( (1-v_0^2) (u^0)^2 - v_0^2 g^{00}) + X (2 u^x u^0 (1-v_0^2) - 2 v_0^2 g^{x0}) + (1-v_0^2) (u^x)^2 - v_0^2 g^{xx} \\
a &= (1-v_0^2) (u^0)^2 - v_0^2 g^{00} = (1-v_0^2) (u^0)^2 + v_0^2/\alpha^2 \leftarrow {\rm VERIFIED} \\
b &= 2 u^x u^0 (1-v_0^2) - 2 v_0^2 \beta^x/\alpha^2 \leftarrow {\rm VERIFIED,\ } X\rightarrow -X, {\rm because\ } X = -w/k_1, {\rm \ and\ we\ are\ solving\ for} -X. \\
c &= (1-v_0^2) (u^x)^2 - v_0^2 (g^{xx}\psi^{-4} - (\beta^x/\alpha)^2) \leftarrow {\rm VERIFIED} \\
v_0^2 &= v_A^2 + c_s^2 (1 - v_A^2) \\
\end{align}



<a id='latex_pdf_output'></a>

# Step 4: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.pdf](Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [24]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb to latex
[NbConvertApp] Writing 98741 bytes to Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
